In [3]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
import pandas as pd

In [4]:
item = pd.read_csv('./상품데이터.csv', engine = 'python', encoding='cp949')
user = pd.read_csv('./사용자데이터.csv', engine = 'python', encoding='cp949')
order = pd.read_csv('./구매데이터.csv', engine = 'python', encoding='cp949')
view = pd.read_csv('./조회기록.csv', engine = 'python', encoding='cp949')
jjim = pd.read_csv('./찜데이터.csv', engine = 'python', encoding='cp949')

In [5]:
view_data = view[['USER_ID', 'ITEM_ID','VIEW_DT']]
view_data = pd.merge(view_data,item,on=['ITEM_ID'],how='left')
view_data = view_data[['USER_ID', 'ITEM_ID', 'ITEM_NAME', 'SALE_PRICE', 'CATEGORY', 'KURLY_HCT', 'KURLY_MCT', 'IS_FOOD', 'ITEM_FEATURE1', 'ITEM_FEATURE2', 'MEANING_SCORE','VIEW_DT']]

In [6]:
# 찜 개수 추가
jjim['JJIM_CNT'] = 1
view_data = pd.merge(view_data, jjim, on=['USER_ID', 'ITEM_ID','ITEM_NAME'], how='left')

In [7]:
# 구매 횟수 추가
order_data=order.groupby(['USER_ID','ITEM_ID'], as_index=False).count()[['USER_ID','ITEM_ID','ITEM_NAME']]
order_data.rename(columns={'ITEM_NAME':'BUY_CNT'},inplace=True)
view_data = pd.merge(view_data, order_data, on=['USER_ID', 'ITEM_ID'], how='left')

In [8]:
# 조회한지 얼마나 지났는지
from datetime import datetime
import datetime as dt

now  = datetime.now()
diff = now - pd.to_datetime(view_data['VIEW_DT'], format="%Y-%m-%d %H:%M:%S")
view_data['VIEW_BEFORE'] = diff.dt.days

In [9]:
#NA값 처리
view_data['MEANING_SCORE']=view_data['MEANING_SCORE'].fillna(0)
view_data['JJIM_CNT']=view_data['JJIM_CNT'].fillna(0)
view_data['BUY_CNT']=view_data['BUY_CNT'].fillna(0)
view_data=view_data.fillna('')

In [10]:
## 데이터셋 생성
view_data['VIEW_CNT'] = 1
dataset=view_data[['USER_ID', 'ITEM_ID', 'VIEW_CNT', 'JJIM_CNT','BUY_CNT','VIEW_BEFORE']]

In [11]:
dataset=dataset.copy()
dataset['RE_VIEW_BEFORE']=np.where(dataset['VIEW_BEFORE']>0, dataset['VIEW_BEFORE'],0.8)

In [12]:
#찜, 구매에 가중치 주기
dataset=dataset.copy()
dataset['SCORE']=(dataset['VIEW_CNT']+dataset['JJIM_CNT']+dataset['BUY_CNT'])/dataset['RE_VIEW_BEFORE']
dataset

,USER_ID,ITEM_ID,VIEW_CNT,JJIM_CNT,BUY_CNT,VIEW_BEFORE,RE_VIEW_BEFORE,SCORE
0,kurlyholic,9079954968,1,1.0,2.0,124,124.0,0.032258
1,kurlyholic,27715132325,1,0.0,1.0,36,36.0,0.055556
2,kurlyholic,82609472025,1,1.0,2.0,196,196.0,0.020408
3,kurlyholic,82609472025,1,1.0,2.0,123,123.0,0.032520
4,kurlyholic,9079954968,1,1.0,2.0,240,240.0,0.016667
...,...,...,...,...,...,...,...,...
75007,QQlNIWiCayc,27020694992,1,0.0,2.0,222,222.0,0.013514
75008,Qp6pyL,373148857,1,0.0,0.0,316,316.0,0.003165
75009,Jac2rwryU,473219068,1,0.0,0.0,160,160.0,0.006250
75010,CkJXFDQ,31803364246,1,0.0,0.0,41,41.0,0.024390


# SGD

In [13]:

class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """
        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose
    
    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """
        
        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))
        
        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])
        
        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):
            # rating이 존재하는 index를 기준으로 training
            xi, yi = self._R.nonzero()
            
            for i, j in zip(xi, yi):
                self.gradient_descent(i, j, self._R[i,j])
            cost = self.cost()
            self._training_process.append((epoch, cost))
            
            # print status
            if self._verbose == True and ((epoch+1)%10 == 0):
                print('Iteration: %d ; cost = %.4f' % (epoch+1, cost))
                
    def update(self, R, epochs):
        """
            R이 업데이트되는 경우 재학습하기 위한 모듈
        """
        # train while epochs
        for epoch in range(epochs):
            xi, yi = R.nonzero()
            
            for i, j in zip(xi, yi):
                self.gradient_descent(i, j, R[i, j])
            cost = self.cost()
            
            # print status
            if self._verbose == True and ((epoch+1)%1 == 0):
                print('Iteration: %d ; cost = %.4f' % (epoch+1, cost))
        
    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """
        # xi, yi: R[xi, yi]는 nonzero인 value를 의미
        xi, yi = self._R.nonzero()
        cost = 0
        
        for x, y in zip(xi, yi):
            cost += pow(self._R[x,y] - self.get_prediction(x,y), 2)
            
        return np.sqrt(cost/len(xi))
    
    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """
        dp = (error * self._Q[j,:]) - (self._reg_param * self._P[i,:])
        dq = (error * self._P[i,:]) - (self._reg_param * self._Q[j,:])
        return dp, dq
    
    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """
        
        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction
        
        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])
        
        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq
        
    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j,:].T)
    
    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:,]+self._P.dot(self._Q.T)

In [14]:
item_matrix = dataset[['USER_ID', 'ITEM_ID','SCORE']].pivot_table(columns=['ITEM_ID'], index='USER_ID', values='SCORE', aggfunc='sum')
item_matrix = item_matrix.fillna(0)
user_id = item_matrix.index
item_matrix.reset_index(inplace=True, drop=True)
item_matrix

ITEM_ID,240171,268969,269719,300718,302519,337015,552431,593297,594006,653560,...,84402463245,84402746117,84402746276,84402747146,84402747325,84403638069,84403819984,84404055313,84404165525,84404293908
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
R = np.array(item_matrix)

In [16]:
factorizer = MatrixFactorization(R, k=20, learning_rate = 0.01, reg_param=0.01, epochs=50, verbose =True)
factorizer.fit()

Iteration: 10 ; cost = 0.3920
Iteration: 20 ; cost = 0.2183
Iteration: 30 ; cost = 0.1586
Iteration: 40 ; cost = 0.1287
Iteration: 50 ; cost = 0.1102


In [17]:
# factorizer.update(R=R, epochs=5)

In [18]:
result = factorizer.get_complete_matrix()
result[0]

array([ 1.23419022, -0.32850235,  1.61912285, ..., -0.71250492,
       -0.03969792, -2.29300748])

In [19]:
# 사용자 번호
user_num = 3

item_list = item_matrix.columns.tolist()
item_idx = [x for x in range(len(item_list))]
item_score = dict(zip(item_list, result[user_num]))
item_score

{240171: 0.6340322839587296,
 268969: -0.053281657306554125,
 269719: 0.43730308615358643,
 300718: 0.7435100554353707,
 302519: -0.8501302328594702,
 337015: 0.7461411764192417,
 552431: 0.4174356744921508,
 593297: 0.18024061310793435,
 594006: -0.7152292528985269,
 653560: 1.5440094140293759,
 745198: -0.15349878744482337,
 1018337: 0.2688133020690826,
 1509927: 1.264791032291245,
 1578163: 0.29559644228723936,
 1787258: -0.11626412495042437,
 1997565: -0.24809205077572974,
 2187157: -0.044282562337837617,
 2187256: 0.040955253973360035,
 2350746: 0.7568595153368507,
 3012567: 0.05407494746574133,
 3572268: 0.5431727110266565,
 3633250: 0.0033513175075882418,
 3755166: 0.5194392706835296,
 3814535: -0.20124785231591685,
 3822346: 0.6925138471643333,
 4156977: 0.5424362505536905,
 4190875: -0.8850348073187017,
 4528715: -1.4361558290945484,
 4808773: 0.6056237452810196,
 5419283: -0.3053550435307637,
 5820252: 1.2138818323873162,
 6160366: 0.5184435861046199,
 6162193: 0.037232562213

In [20]:
user1_rec = pd.DataFrame(list(item_matrix.iloc[user_num].T), index=item_matrix.iloc[user_num].T.index,columns=['SCORE'])
user1_view = user1_rec[user1_rec['SCORE'] != 0]
user1_view = pd.merge(user1_view, item, on='ITEM_ID', how='left')
user1_view

,ITEM_ID,SCORE,CATEGORY,ITEM_NAME,SALE_PRICE,REDUCED_PRICE,BENEFIT_PRICE,DISCOUNT_RATE,IS_FOOD,ITEM_FEATURE1,ITEM_FEATURE2,MEANING_SCORE,PRICE_DT,KURLY_HCT,KURLY_MCT
0,81486497,0.125000,혼합조미료,미원 쇠고기 진국 다시 1.1KG,"9,480","9,480",0,-,식품,NaN,NaN,NaN,2022-07-01,면_양념_오일,소금_설탕_향신료
1,151176461,0.003559,색조화장품,독일 디엠 Belea 발레아 립 케어 인텐시브 4.8g,"4,100","4,100",0,-,비식품,NaN,NaN,NaN,2022-07-01,스킨케어_메이크업,메이크업
2,352096241,0.005155,생선통조림,유동 어부지리 새꼬막간장비빔 164G,"5,990","5,990",0,-,식품,NaN,NaN,NaN,2022-07-01,수산_해산_건어물,수산가공품
3,564039737,0.200000,필기구,오만과 편견 책갈피 볼펜 세트-자화상,"3,000","2,700",-300,0.1,비식품,NaN,NaN,NaN,2022-07-01,생활용품_리빙_캠핑,생활잡화_문구
4,780172450,0.028571,헬스기구,굿프렌드 줄리엔강 굿터치진동운동기 DQ-004 화이트,"293,360","293,360",0,-,비식품,NaN,NaN,NaN,2022-07-01,가전제품,헬스기구
5,974897067,0.008969,방향제,방향제 깨끗한집 고급 쟈스민 X ( 4매입 ),"8,990","8,990",0,-,비식품,NaN,NaN,NaN,2022-07-01,생활용품_리빙_캠핑,세제_청소용품
6,1079796170,0.002817,헬스기구,바디스컬쳐 저항밴드3개입,"20,300","20,300",0,-,비식품,NaN,NaN,NaN,2022-07-01,가전제품,헬스기구
7,1211561758,0.008264,아동복,데이즈 [The 편한] 주니어 스포츠 1단계 브라,"12,800","12,800",0,-,비식품,NaN,NaN,NaN,2022-07-01,베이비_키즈_완구,아동패션
8,1291774671,0.005495,아동복,주니어 1단계 텐셀원단 브라 AA컵 cbr6057,"6,600","6,600",0,-,비식품,NaN,NaN,NaN,2022-07-01,베이비_키즈_완구,아동패션
9,1308031321,0.007143,복사용지,플라잉칼라 칼라용지 컬러용지 복사용지 A4 R01 10색혼합 80g 250매,"14,350","14,350",0,-,비식품,NaN,NaN,NaN,2022-07-01,생활용품_리빙_캠핑,생활잡화_문구


In [21]:
# 각 사용자에 대해서 상품 추천
recommends = sorted(item_score, key=lambda x:item_score[x], reverse=True)
recommends = pd.DataFrame(columns = ['ITEM_ID'], data=recommends)

## 첫번째 사용자에 대한 추천 결과
recommends = pd.merge(recommends, item, on='ITEM_ID', how='left')
recommends_item = recommends[:20]
recommends_item

,ITEM_ID,CATEGORY,ITEM_NAME,SALE_PRICE,REDUCED_PRICE,BENEFIT_PRICE,DISCOUNT_RATE,IS_FOOD,ITEM_FEATURE1,ITEM_FEATURE2,MEANING_SCORE,PRICE_DT,KURLY_HCT,KURLY_MCT
0,31480385418,설탕,베트남 락슈가 등펜 3K 얼음설탕 500g,"3,960","3,960",0,-,식품,NaN,NaN,NaN,2022-07-01,면_양념_오일,소금_설탕_향신료
1,82603897187,커피,[칸타타]원두커피(프리미엄 라떼) 390ml,"2,800","2,800",0,-,식품,NaN,NaN,NaN,2022-07-01,생수_음료_우유_커피,커피
2,1308022108,스낵과자,(1703870) 롯데 꼬깔콘 치토스 4번들x8묶음,"31,390","31,390",0,-,식품,NaN,NaN,NaN,2022-07-01,간식_과자_떡,과자_스낵_쿠키
3,1894049962,필기구,아트박스/모나미 모나미 예감적중 2IN1 컴퓨터용사인펜[00032320],400,400,0,-,비식품,NaN,NaN,NaN,2022-07-01,생활용품_리빙_캠핑,생활잡화_문구
4,25429455602,냉동식품,롯데푸드 옛날군만두 1kg,"3,720","3,720",0,-,식품,NaN,NaN,NaN,2022-07-01,간식_과자_떡,아이스크림
5,82539601210,쌀,[특등급] 임금님표 이천쌀 알찬미 10KG(포),"39,900","39,900",0,-,식품,NaN,NaN,NaN,2022-07-01,과일_견과_쌀,쌀_잡곡
6,28256479010,낙지,프레시원 냉동 절단낙지 (M) 500g x4개 냉동낙지,"57,180","57,180",0,-,식품,NaN,NaN,NaN,2022-07-01,수산_해산_건어물,오징어_낙지_문어
7,687749593,회화용구,파스텔 오일파스텔 PP CASE 펜텔 오일 COLOR GHT-12 12 X ( 2매입 ),"9,940","9,940",0,-,비식품,NaN,NaN,NaN,2022-07-01,생활용품_리빙_캠핑,생활잡화_문구
8,1376217820,필기구,제트스트림 SXN-150 0.7mm 볼펜 (낱개),"1,600","1,600",0,-,비식품,NaN,NaN,NaN,2022-07-01,생활용품_리빙_캠핑,생활잡화_문구
9,32710878469,파,lalavlamall_전라도 남도 별미 쪽파 김치 파김치 1kg,"14,060","14,060",0,-,식품,NaN,NaN,NaN,2022-07-01,채소,양파_대파_마늘_배추


## SGD 평가

In [22]:
y_true = np.array(item_matrix)
y_pred = result

from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(y_true, y_pred)
print('MAE :', MAE)

from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_true, y_pred)
RMSE = mean_squared_error(y_true, y_pred)**0.5
print('MSE :', MSE)
print('RMSE :', RMSE)

MAE : 1.0321598683225965
MSE : 2.1913830968889068
RMSE : 1.480332090069288


### 1) 조회, 찜, 구매 각각 가중치 1 (횟수)
- MSE : 13.938269199664816
- RMSE : 3.733399148184509

### 2) 조회, 찜, 구매 각각 가중치 1,2,3 (횟수)
- MSE : 20.485551881166224
- RMSE : 4.5260967600313435

### 3) 조회, 찜, 구매 각각 가중치 1 (횟수) / 최근 조회 일자(일수, 0보다 작을때 0.8)
- MSE : 11.23722465926306
- RMSE : 3.352196989925124

-- 이외에 가중치를 1:1.5:2로 주거나, 조회일수차이를 0.8보다 키우거나 줄였을 때 모든 경우에서 3)보다 결과가 좋지 않음

### <span style="color:red"> 4) 조회, 찜, 구매 데이터 3배 증가 </span> >> 채택
- MAE : 1.0321598683225965
- MSE : 2.1913830968889068
- RMSE : 1.480332090069288

# pickle 파일 추출

In [23]:
import pickle

In [24]:
user_dic = dict(zip(user_id, item_matrix.index))
user_dic

with open('./user_dic.pkl', 'wb') as f:
    pickle.dump(user_dic, f)

In [25]:
item_dic = dict(zip(item_list, item_idx))
item_dic

with open('./item_dic.pkl', 'wb') as f:
    pickle.dump(item_dic, f)

In [26]:
with open('./model.pkl', 'wb') as f:
    pickle.dump(factorizer, f)

# 카테고리 추천 모델

In [27]:
# 추천 상위 200개 상품 중 최다품목 10가지
best_catagory=recommends[:200]['CATEGORY'].value_counts()[:10]
best_catagory=pd.DataFrame(columns = ['CATEGORY'], data=best_catagory.index)
best_catagory_list=best_catagory['CATEGORY'].tolist()
best_catagory_list

['필기구', '아동복', '냉동식품', '스낵과자', '즉석식품', '기초화장품', '생선통조림', '등산복', '점퍼', '티셔츠']

In [28]:
# 카테고리별 주문횟수가 많은 상품 20개씩 추출
order_catagory=pd.merge(order, item, on=['ITEM_ID', 'ITEM_NAME'], how='left')[['CATEGORY','ITEM_ID','ITEM_NAME']]
order_catagory['COUNT']=1
order_catagory=order_catagory.groupby(['CATEGORY','ITEM_ID','ITEM_NAME'], as_index=False).sum()

In [29]:
# 추천품목에 해당하는 top20 상품
order_catagory_top=order_catagory.sort_values(by="COUNT", ascending=False).groupby("CATEGORY").head(20)
order_catagory_top=order_catagory_top.sort_values(by=["CATEGORY","COUNT"], ascending=[True, False])
order_catagory_top=order_catagory_top.loc[order_catagory_top['CATEGORY'].isin(best_catagory_list)]
order_catagory_top

,CATEGORY,ITEM_ID,ITEM_NAME,COUNT
1799,기초화장품,895451870,달팡 인트랄 수딩 크림 50ml,4
1788,기초화장품,671720573,오 떼르말 150ml 1+1 기획,4
1759,기초화장품,341293999,크리니크 - 리페어웨어 스컬프팅 나이트 크림,4
1947,기초화장품,32863864834,글랜무어 포어 포졸라닉 스크럽 &amp; 폼 클렌저 100ml,4
1965,기초화장품,33206669922,퓨어 유자 클렌징 프리비아 430ml 마사지크림Harpazo,4
...,...,...,...,...
15886,필기구,1297199680,보드마카 화이트 사무용 검정빨강세트 선명한색상 X ( 2매입 ),4
15894,필기구,1344698726,각인)유니 쿠루토가 스탠다드 0.5mm 1개입 M5-450,4
15856,필기구,1230823374,샤프 샤프추천 좋은샤프 신와 샤프연필심적색 연필심 S-78474 389-0005 X...,4
15981,필기구,1894075972,아트박스/스테들러 스테들러 노리스클럽 557 10 수학교구 세트[00137056],4


In [30]:
category_rec_dict = {}
for cat in best_catagory_list:
    item_id_list = list(order_catagory_top[order_catagory_top['CATEGORY']==cat]['ITEM_ID'])
    category_rec_dict[cat] = item_id_list

category_rec_dict

{'필기구': [733404468,
  473219068,
  1084731208,
  733397491,
  420376944,
  2012651576,
  1282703812,
  1168649056,
  341269856,
  409385083,
  384114161,
  1895008613,
  1870991758,
  1715724370,
  1586800402,
  1297199680,
  1344698726,
  1230823374,
  1894075972,
  729914830],
 '아동복': [870357289,
  1406689724,
  509619622,
  786472706,
  188449313,
  236760281,
  72334918,
  1841309607,
  1896719331,
  1793944225,
  2067868808,
  32222576191,
  2142159079,
  2121612160,
  2104660094,
  1400256887,
  1390983137,
  1582541660,
  1551813050,
  56797938],
 '냉동식품': [1282014568,
  1105510480,
  2135423121,
  1103928550,
  1390054592,
  1135211114,
  1197140970,
  29886583559,
  421240017,
  1312379094,
  533710529,
  458291321,
  1222250834,
  1197147722,
  24088071745,
  1197207248,
  26224641613,
  1263109006,
  1104672494,
  31008158522],
 '스낵과자': [1107562062,
  1308022048,
  1137143846,
  32999448261,
  1105620875,
  33079464306,
  1104850244,
  1103927775,
  1104850249,
  1219133483,


In [31]:
import json

json.dumps(category_rec_dict, ensure_ascii = False)

'{"필기구": [733404468, 473219068, 1084731208, 733397491, 420376944, 2012651576, 1282703812, 1168649056, 341269856, 409385083, 384114161, 1895008613, 1870991758, 1715724370, 1586800402, 1297199680, 1344698726, 1230823374, 1894075972, 729914830], "아동복": [870357289, 1406689724, 509619622, 786472706, 188449313, 236760281, 72334918, 1841309607, 1896719331, 1793944225, 2067868808, 32222576191, 2142159079, 2121612160, 2104660094, 1400256887, 1390983137, 1582541660, 1551813050, 56797938], "냉동식품": [1282014568, 1105510480, 2135423121, 1103928550, 1390054592, 1135211114, 1197140970, 29886583559, 421240017, 1312379094, 533710529, 458291321, 1222250834, 1197147722, 24088071745, 1197207248, 26224641613, 1263109006, 1104672494, 31008158522], "스낵과자": [1107562062, 1308022048, 1137143846, 32999448261, 1105620875, 33079464306, 1104850244, 1103927775, 1104850249, 1219133483, 17162804194, 24991787522, 1525569029, 31192555674, 1975804824, 30521036647, 30521003979, 30520945001, 31552953581, 84348943138], "즉석식품